In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from keras.utils.np_utils import to_categorical
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, Flatten, Lambda, Activation,BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam

train_df = pd.read_json('../input/train.json')
test_df = pd.read_json('../input/test.json')

Using TensorFlow backend.


In [2]:
def get_image(df):
    '''Create 3-channel 'images'. Return rescale-normalised images.'''
    images = []
    for i, row in df.iterrows():
        # Formulate the bands as 75x75 arrays
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = (band_1 + band_2)/2

        # Rescale
        r = (band_1 - band_1.min()) / (band_1.max() - band_1.min())
        g = (band_2 - band_2.min()) / (band_2.max() - band_2.min())
        b = (band_3 - band_3.min()) / (band_3.max() - band_3.min())

        rgb = np.dstack((r, g, b))
        images.append(rgb)
    return np.array(images)


train_x = get_image(train_df)
test_x = get_image(test_df)

print(train_x.shape,test_x.shape)

(1604, 75, 75, 3) (8424, 75, 75, 3)


In [3]:
y = train_df.is_iceberg.values
print(y[:5])

[0 0 1 0 0]


In [4]:
def rot_aut(Xtr,ytr):
    # aug on train
    data_cnt = len(ytr)
    print(data_cnt)
    aug_X = []
    aug_y = []

    for i in range(data_cnt):
        img = Xtr[i]
        tmp_y = ytr[i]

        # org img
        aug_X.append(img)
        aug_y.append(tmp_y)

        # flip
        tmp_img = np.fliplr(img)
        aug_X.append(tmp_img)
        aug_y.append(tmp_y)

        tmp_img = np.flipud(img)
        aug_X.append(tmp_img)
        aug_y.append(tmp_y)

        tmp_img = np.rot90(img)
        aug_X.append(tmp_img)
        aug_y.append(tmp_y)
    return np.array(aug_X),np.array(aug_y)

In [5]:
from keras.callbacks import ModelCheckpoint
from keras.layers import Input,AveragePooling2D,GlobalMaxPooling2D,GlobalAveragePooling2D,Flatten
from keras import layers
from keras.models import Model



def create_model():
    img_input = Input(shape=(75,75,3))
    channel_axis = 3
    
    # Block 1
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    x = Flatten()(x)
    x = Dense(256,activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(1,activation='sigmoid')(x)
    
    return Model(img_input, x, name='inception_v3')
print('model model')

test_m = create_model()
test_m.summary()

model model
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 75, 75, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 75, 75, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 75, 75, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 37, 37, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 37, 37, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 37, 37, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 18, 18, 128)       0        

In [6]:
from sklearn.model_selection import KFold
def kfold_train(fold_cnt=3,rnd=233):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        curr_x,curr_y = rot_aut(curr_x,curr_y)
        val_x,val_y = train_x[test_index],y[test_index]
        
        model = create_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        model.fit(curr_x,curr_y,
                  validation_data=(val_x,val_y),
                  batch_size=32, epochs=20, 
                  verbose=2,
                  callbacks=[model_chk]
                 )
        model = load_model(model_p)
        
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=4)

1203
Train on 4812 samples, validate on 401 samples
Epoch 1/20
Epoch 00001: val_loss improved from inf to 0.86120, saving model to best_m.h5
 - 19s - loss: 0.6848 - acc: 0.5468 - val_loss: 0.8612 - val_acc: 0.5362
Epoch 2/20
Epoch 00002: val_loss improved from 0.86120 to 0.58815, saving model to best_m.h5
 - 17s - loss: 0.5972 - acc: 0.6793 - val_loss: 0.5881 - val_acc: 0.6708
Epoch 3/20
Epoch 00003: val_loss did not improve
 - 17s - loss: 0.5795 - acc: 0.6906 - val_loss: 0.6179 - val_acc: 0.6234
Epoch 4/20
Epoch 00004: val_loss improved from 0.58815 to 0.52427, saving model to best_m.h5
 - 17s - loss: 0.5525 - acc: 0.6962 - val_loss: 0.5243 - val_acc: 0.6933
Epoch 5/20
Epoch 00005: val_loss improved from 0.52427 to 0.51833, saving model to best_m.h5
 - 17s - loss: 0.5180 - acc: 0.7147 - val_loss: 0.5183 - val_acc: 0.7132
Epoch 6/20
Epoch 00006: val_loss improved from 0.51833 to 0.47326, saving model to best_m.h5
 - 17s - loss: 0.4733 - acc: 0.7560 - val_loss: 0.4733 - val_acc: 0.7082


Epoch 18/20
Epoch 00018: val_loss did not improve
 - 17s - loss: 0.2144 - acc: 0.9065 - val_loss: 0.3040 - val_acc: 0.8728
Epoch 19/20
Epoch 00019: val_loss improved from 0.24200 to 0.23699, saving model to best_m.h5
 - 17s - loss: 0.2211 - acc: 0.9071 - val_loss: 0.2370 - val_acc: 0.8978
Epoch 20/20
Epoch 00020: val_loss did not improve
 - 17s - loss: 0.2276 - acc: 0.9075 - val_loss: 0.2534 - val_acc: 0.9052
1203
Train on 4812 samples, validate on 401 samples
Epoch 1/20
Epoch 00001: val_loss improved from inf to 0.90219, saving model to best_m.h5
 - 20s - loss: 0.6722 - acc: 0.5779 - val_loss: 0.9022 - val_acc: 0.5885
Epoch 2/20
Epoch 00002: val_loss improved from 0.90219 to 0.58886, saving model to best_m.h5
 - 17s - loss: 0.6141 - acc: 0.6527 - val_loss: 0.5889 - val_acc: 0.6434
Epoch 3/20
Epoch 00003: val_loss did not improve
 - 18s - loss: 0.5389 - acc: 0.7255 - val_loss: 1.0742 - val_acc: 0.7332
Epoch 4/20
Epoch 00004: val_loss improved from 0.58886 to 0.49867, saving model to be

In [7]:
import pickle
with open('../features/vgg16_1_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
from sklearn.metrics import log_loss
print(log_loss(y,train_pred))

submission = pd.DataFrame()
submission['id']=test_df['id']
submission['is_iceberg']=test_pred
print(submission.head())
submission.to_csv('../results/vgg16_1_sub.csv', index=False)

0.286317592196
         id  is_iceberg
0  5941774d    0.538029
1  4023181e    0.415186
2  b20200e4    0.051987
3  e7f018bb    0.988152
4  4371c8c3    0.379571
